# Import Libraries

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re

# Property Page List

In [2]:
#main page
base_page = 'https://www.bukitvista.com/property'

#empty list
page_urls = []

#URLS for each page
num_pages = 12

#looping for list of base_page + page
for page_num in range (1, num_pages + 1):
  page_url = f'{base_page}/page/{page_num}'
  page_urls.append(base_page if page_num == 1 else page_url)

In [3]:
page_urls

['https://www.bukitvista.com/property',
 'https://www.bukitvista.com/property/page/2',
 'https://www.bukitvista.com/property/page/3',
 'https://www.bukitvista.com/property/page/4',
 'https://www.bukitvista.com/property/page/5',
 'https://www.bukitvista.com/property/page/6',
 'https://www.bukitvista.com/property/page/7',
 'https://www.bukitvista.com/property/page/8',
 'https://www.bukitvista.com/property/page/9',
 'https://www.bukitvista.com/property/page/10',
 'https://www.bukitvista.com/property/page/11',
 'https://www.bukitvista.com/property/page/12']

In [4]:
property_urls = []

for page_num in page_urls:
  page_url = f'{page_num}'
  req = requests.get(page_url)
  soup = BeautifulSoup(req.content, 'html.parser')

  url = soup.find_all('h2', {'class': "item-title"})

  for a in url:
      property_urls.append(a.find_all('a', href=True)[0]['href'])



property_urls

['https://www.bukitvista.com/property/artistic-bali-retreat-with-serene-breezes',
 'https://www.bukitvista.com/property/an-artist-cottage-haven-w-serene-pool-in-uluwatu',
 'https://www.bukitvista.com/property/ungasan-chic-villa-with-pool-and-rooftop-patio',
 'https://www.bukitvista.com/property/artful-bungalow-in-uluwatu',
 'https://www.bukitvista.com/property/whispers-of-bliss-2br-tropical-oasis-in-nusa-dua',
 'https://www.bukitvista.com/property/3br-art-deco-villa-with-pool-garden-oasis',
 'https://www.bukitvista.com/property/5-bedroom-villa-for-leasehold-in-umalas',
 'https://www.bukitvista.com/property/retreat-to-nature-jungle-house-villa-near-canggu',
 'https://www.bukitvista.com/property/luxurious-private-villa-with-pool-in-idyllic-pecatu-retreat',
 'https://www.bukitvista.com/property/peaceful-villa-surrounded-by-rice-fields-in-canggu',
 'https://www.bukitvista.com/property/secluded-2br-villa-w-pool-complex-in-nusa-dua',
 'https://www.bukitvista.com/property/modern-breezy-homey-

In [5]:
data_url = []

def object(key, value):
  return {key: value}

In [6]:
title = {'Name': soup.find_all('title')[0].text}


In [7]:
for page in property_urls:
    page_url = f'{page}'
    req = requests.get(page_url)
    soup = BeautifulSoup(req.content, 'html.parser')

    titles = soup.find_all('title')
    if titles:
        title = {'Name': titles[0].text}
    else:
        title = {'Name': 'No title found'}

    page_data = soup.find_all('ul', {'class': "list-1-cols list-unstyled"})

    if page_data:
        for d in page_data[0].select('li'):
            columnName = d.select('strong')[0].text
            columnNameClean = columnName.replace(":", "")

            columnValue = d.select('span')[0].text

            title.update(object(columnNameClean, columnValue))

    data_url.append(title)

data_url

[{'Name': 'Artistic Bali Retreat with Serene Breezes - Bukit Vista',
  'Property ID': 'PR0792',
  'Price': 'Start from $2,400 USD per Month',
  'Bedrooms': '2',
  'Rooms': '4',
  'Bathroom': '1',
  'Property Type': 'Guest House, Residential',
  'Property Status': 'Bali Vacation Rental, Instagramable Stay, Long Stay Trend Property',
  'Guest Number': '4'},
 {'Name': 'An Artist Cottage Haven W/ Serene Pool in Uluwatu - Bukit Vista',
  'Property ID': 'PR0792',
  'Price': 'Start from $1,400 USD per Month',
  'Bedroom': '1',
  'Rooms': '2',
  'Bathroom': '1',
  'Property Type': 'Villa, Residential',
  'Property Status': 'Bali Vacation Rental, Instagramable Stay, Long Stay Trend Property',
  'Guest Number': '2'},
 {'Name': 'Ungasan Chic Villa with Pool and Rooftop Patio - Bukit Vista',
  'Property ID': 'PR0777',
  'Price': 'Start from $1,900 USD per Month',
  'Bedrooms': '3',
  'Rooms': '6',
  'Bathrooms': '3',
  'Garage': '1',
  'Property Type': 'Villa, Residential',
  'Property Status': 'B

In [8]:
len(data_url)

92

In [9]:
df = pd.DataFrame.from_dict(data_url)
df

,Name,Property ID,Price,Bedrooms,Rooms,Bathroom,Property Type,Property Status,Guest Number,Bedroom,Bathrooms,Garage,Property Size,Land Area,Year Built,Room
0,Artistic Bali Retreat with Serene Breezes - Bu...,PR0792,"Start from $2,400 USD per Month",2,4,1,"Guest House, Residential","Bali Vacation Rental, Instagramable Stay, Long...",4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,An Artist Cottage Haven W/ Serene Pool in Uluw...,PR0792,"Start from $1,400 USD per Month",NaN,2,1,"Villa, Residential","Bali Vacation Rental, Instagramable Stay, Long...",2,1,NaN,NaN,NaN,NaN,NaN,NaN
2,Ungasan Chic Villa with Pool and Rooftop Patio...,PR0777,"Start from $1,900 USD per Month",3,6,NaN,"Villa, Residential","Bali Vacation Rental, Group Friendly Stay, Lon...",6,NaN,3,1,NaN,NaN,NaN,NaN
3,Artful Bungalow Nestled in Lush Beauty Uluwatu...,PR0792,"Start from $1,400 USD per Month",NaN,2,1,"Villa, Residential","Bali Vacation Rental, Instagramable Stay, Long...",2,1,NaN,1,NaN,NaN,NaN,NaN
4,Whispers of Bliss: 2BR Tropical Oasis in Nusa ...,PR0797,"Start from $1,800 USD per Month",2,4,NaN,"Villa, Residential","Bali Vacation Rental, Long Stay Trend Property...",4,NaN,2,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,Villa 4 Kamar di Uluwatu oleh Bukit Vista - Bu...,42885147,Start from Rp 28.000.000/Month,4,4,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN
88,4BR Homey Villa in Uluwatu | Kitchen + Wifi + ...,PR0528,"Start from $4,930 USD per Month",4,4,NaN,Villa,Top Trending,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN
89,Bukit Vista | Santorini Surfer Loft in Uluwatu...,31152138,"Start from $1,390 USD per Month",NaN,NaN,1,"Guest House, Style",Top Trending,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
90,Pondok Di Tengah Hutan dengan Suasana Tenang d...,36539552,Start from Rp 6.000.000/Month,NaN,NaN,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Name             92 non-null     object
 1   Property ID      85 non-null     object
 2   Price            90 non-null     object
 3   Bedrooms         45 non-null     object
 4   Rooms            30 non-null     object
 5   Bathroom         48 non-null     object
 6   Property Type    78 non-null     object
 7   Property Status  73 non-null     object
 8   Guest Number     60 non-null     object
 9   Bedroom          46 non-null     object
 10  Bathrooms        43 non-null     object
 11  Garage           5 non-null      object
 12  Property Size    2 non-null      object
 13  Land Area        1 non-null      object
 14  Year Built       3 non-null      object
 15  Room             6 non-null      object
dtypes: object(16)
memory usage: 11.6+ KB


In [11]:
kolom_integer = ['Bedrooms', 'Rooms', 'Bathroom', 'Guest Number', 'Bedroom', 'Bathrooms', 'Garage', 'Year Built', 'Room']

df[kolom_integer] = df[kolom_integer].astype(float)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             92 non-null     object 
 1   Property ID      85 non-null     object 
 2   Price            90 non-null     object 
 3   Bedrooms         45 non-null     float64
 4   Rooms            30 non-null     float64
 5   Bathroom         48 non-null     float64
 6   Property Type    78 non-null     object 
 7   Property Status  73 non-null     object 
 8   Guest Number     60 non-null     float64
 9   Bedroom          46 non-null     float64
 10  Bathrooms        43 non-null     float64
 11  Garage           5 non-null      float64
 12  Property Size    2 non-null      object 
 13  Land Area        1 non-null      object 
 14  Year Built       3 non-null      float64
 15  Room             6 non-null      float64
dtypes: float64(9), object(7)
memory usage: 11.6+ KB


In [13]:
df['Total Rooms'] = df['Room'].combine_first(df['Rooms'])

In [14]:
df['Total Rooms'].fillna(0, inplace=True)

In [15]:
df['Garage'].fillna(0, inplace=True)

In [16]:
df['Total Bedrooms'] = df['Bedroom'].combine_first(df['Bedrooms'])

In [17]:
nan_values = df[df['Total Bedrooms'].isnull()]
print(nan_values)

                      Name Property ID Price  Bedrooms  Rooms  Bathroom  \
24  balibiza - Bukit Vista         NaN   NaN       NaN    NaN       NaN   

   Property Type Property Status  Guest Number  Bedroom  Bathrooms  Garage  \
24           NaN             NaN           NaN      NaN        NaN     0.0   

   Property Size Land Area  Year Built  Room  Total Rooms  Total Bedrooms  
24           NaN       NaN         NaN   NaN          0.0             NaN  


In [18]:
df.dropna(subset=['Total Bedrooms'], inplace=True)

In [19]:
df['Total Bathrooms'] = df['Bathroom'].combine_first(df['Bathrooms'])

In [20]:
columns_to_drop = ['Room', 'Rooms', 'Bedroom', 'Bedrooms', 'Bathroom', 'Bathrooms']
df.drop(columns=columns_to_drop, inplace=True)

In [21]:
df

,Name,Property ID,Price,Property Type,Property Status,Guest Number,Garage,Property Size,Land Area,Year Built,Total Rooms,Total Bedrooms,Total Bathrooms
0,Artistic Bali Retreat with Serene Breezes - Bu...,PR0792,"Start from $2,400 USD per Month","Guest House, Residential","Bali Vacation Rental, Instagramable Stay, Long...",4.0,0.0,NaN,NaN,NaN,4.0,2.0,1.0
1,An Artist Cottage Haven W/ Serene Pool in Uluw...,PR0792,"Start from $1,400 USD per Month","Villa, Residential","Bali Vacation Rental, Instagramable Stay, Long...",2.0,0.0,NaN,NaN,NaN,2.0,1.0,1.0
2,Ungasan Chic Villa with Pool and Rooftop Patio...,PR0777,"Start from $1,900 USD per Month","Villa, Residential","Bali Vacation Rental, Group Friendly Stay, Lon...",6.0,1.0,NaN,NaN,NaN,6.0,3.0,3.0
3,Artful Bungalow Nestled in Lush Beauty Uluwatu...,PR0792,"Start from $1,400 USD per Month","Villa, Residential","Bali Vacation Rental, Instagramable Stay, Long...",2.0,1.0,NaN,NaN,NaN,2.0,1.0,1.0
4,Whispers of Bliss: 2BR Tropical Oasis in Nusa ...,PR0797,"Start from $1,800 USD per Month","Villa, Residential","Bali Vacation Rental, Long Stay Trend Property...",4.0,0.0,NaN,NaN,NaN,4.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,Villa 4 Kamar di Uluwatu oleh Bukit Vista - Bu...,42885147,Start from Rp 28.000.000/Month,NaN,NaN,NaN,0.0,NaN,NaN,NaN,4.0,4.0,4.0
88,4BR Homey Villa in Uluwatu | Kitchen + Wifi + ...,PR0528,"Start from $4,930 USD per Month",Villa,Top Trending,NaN,0.0,NaN,NaN,NaN,4.0,4.0,4.0
89,Bukit Vista | Santorini Surfer Loft in Uluwatu...,31152138,"Start from $1,390 USD per Month","Guest House, Style",Top Trending,NaN,0.0,NaN,NaN,NaN,0.0,1.0,1.0
90,Pondok Di Tengah Hutan dengan Suasana Tenang d...,36539552,Start from Rp 6.000.000/Month,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,1.0,1.0


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91 entries, 0 to 91
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             91 non-null     object 
 1   Property ID      85 non-null     object 
 2   Price            90 non-null     object 
 3   Property Type    78 non-null     object 
 4   Property Status  73 non-null     object 
 5   Guest Number     60 non-null     float64
 6   Garage           91 non-null     float64
 7   Property Size    2 non-null      object 
 8   Land Area        1 non-null      object 
 9   Year Built       3 non-null      float64
 10  Total Rooms      91 non-null     float64
 11  Total Bedrooms   91 non-null     float64
 12  Total Bathrooms  91 non-null     float64
dtypes: float64(6), object(7)
memory usage: 10.0+ KB
